In [1]:
from canvasapi import Canvas

In [8]:
CANVAS_URL = "https://canvas.liverpool.ac.uk"
CANVAS_TOKEN = "15502~Z3rx68ayZ5BDUspm2OoFsmRKNaQOSq3ICYuvZcrsvVkzSRXSusDJ4L8lptQCoJ4w"

In [9]:
canvas = Canvas(CANVAS_URL, CANVAS_TOKEN)

In [10]:
course = canvas.get_course(54932)

In [11]:
enrollments = [x for x in course.get_enrollments(include=["user"])]

In [12]:
# order enrollments by user["sortable_name"]
enrollments.sort(key=lambda x: x.user["sortable_name"])

In [13]:
for e in enrollments:
    print(e.user["sortable_name"])

Acosta-Serrano, Alvaro
Adams, Emily
Alfirevic, Ana
Aljayyoussi, Ghaith
Allen, Fabia
Allison, Heather
Ameh, Charles
Angus, Robert
Antczak, Philipp
Antic, Dragan
Antonyuk, Svetlana
Arrowsmith, Sarah
Atkins, Rachael
Atkinson, David
Auer-Fowler, Anna
Awais, Raheela
Baker, Bex
Ball, Sam
Barclay, Jeff
Barden, Matthew
Barker, Robert
Barlow, Laura
Barr, David
Barraclough, Dong
Barrera Briceno, Lawrence
Barribeau, Seth
Barsukov, Igor
Basenko, Evelina
Bates, Andy
Bates, Karl
Baylis, Matthew
Begon, Michael
Bennett, Daimark
Bentley, Rob
Berenbrink, Michael
Betancourt, Andrea
Beynon, Rob
Biagini, Giancarlo
Blacklock, Liam
Blagrove, Marcus
Blake, Neil
Blundell, Richard
Bollback, Jonathan
Bond, Alistair
Bonnett, Laura
Bosworth, Lucy
Bou-Gharios, George
Boyd, Alan
Bronowski, Christina
Burns, Helen
Burnside, Girvan
Butts, Thomas
Byrne, Rhianne
Caamano-Gutierrez, Eva
Caddick, Mark
Caminade, Cyril
Camp, Ariel
Campbell, Barry
Campos Soares Da Rocha, Sonia
Canniffe, Daniel
Cantley, Siobhan
Cardenas-Serna, 

In [14]:
from fuzzywuzzy import process

def find_most_likely_match(input, names):
    best_match = process.extractOne(input, names)
    return best_match[0]

c:\Users\treharne\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [22]:
names = [e.user["sortable_name"] for e in enrollments]

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("supervisors_2.csv")

In [18]:
df.head()

,last_name,first_name,projects_UG,projects_PG,max_projects,username,email
0,Antonyuk,Svetlana,NaN,NaN,4,NaN,NaN
1,Barclay,Jeff,NaN,NaN,4,NaN,NaN
2,Barsukov,Igor,NaN,NaN,4,NaN,NaN
3,Byrne,Dominic,1.0,1.0,2,NaN,NaN
4,Canniffe,Daniel,NaN,NaN,4,NaN,NaN


In [19]:
# replace NaN in projects_UG with integer value 3
df["projects_UG"] = df["projects_UG"].fillna(3)
df["projects_PG"] = df["projects_PG"].fillna(1)

In [20]:
df.head()

,last_name,first_name,projects_UG,projects_PG,max_projects,username,email
0,Antonyuk,Svetlana,3.0,1.0,4,NaN,NaN
1,Barclay,Jeff,3.0,1.0,4,NaN,NaN
2,Barsukov,Igor,3.0,1.0,4,NaN,NaN
3,Byrne,Dominic,1.0,1.0,2,NaN,NaN
4,Canniffe,Daniel,3.0,1.0,4,NaN,NaN


In [24]:
def get_enrollment_by_sortable_name(name, enrollments):
    for e in enrollments:
        if e.user["sortable_name"] == name:
            return e
    return None

In [33]:
staff = get_enrollment_by_sortable_name("Carleson, Emile", enrollments)
staff.user["sis_user_id"]

AttributeError: 'NoneType' object has no attribute 'user'

In [27]:
# loop through df and find the most likely match for each supervisor
for index, row in df.iterrows():
    try:
        name = f"{row["last_name"]}, {row["first_name"]}"
        match = find_most_likely_match(name, names)
        username = get_enrollment_by_sortable_name(match, enrollments).user["sis_user_id"]
        
        # remove 6 digit number from username
        username = username[6:]
        
        # update df with username
        df.at[index, "username"] = username

        print(f"{name} -> {match} -> {username}")
    except:
        print(f"Error with {name}")

Antonyuk, Svetlana -> Antonyuk, Svetlana -> antonyuk
Barclay, Jeff -> Barclay, Jeff -> barclayj
Barsukov, Igor -> Barsukov, Igor -> igb2
Byrne, Dominic -> Byrne, Rhianne -> rbyrne
Canniffe, Daniel -> Canniffe, Daniel -> canniffe
Chetwynd, Andrew -> Owen, Andrew -> aowen
Clague, Mike -> Clague, Michael -> clague
Darling , Nicola -> Evans, Nicholas -> evansnj
Dart, Caroline -> Dart, Caroline -> cdart
Del Carratore, Francesco -> Del Carratore, Francesco -> mqbpqfd2
Dunn, Rick -> Dunn, Warwick -> dunnw
Emmott, Edward -> Emmott, Edward -> edemmott
Eyers, Patrick -> Eyers, Pat -> paeyers
Eyers, Claire -> Tierney, Claire -> ctierney
Fernig, David -> Fernig, David -> dgfernig
Goodacre, Roy -> Goodacre, Roy -> roygood
Hartwell, James -> Hartwell, James -> hartwell
Hasnain, Samar -> Das, Shampa -> shampa
Helassa, Nordine -> Helassa, Nordine -> nhelassa
Jones, Andrew -> Jones, Andy -> jonesar
Kell, Douglas -> Ke, Youqiang -> yqk
Kenneth, Niall -> Kenneth, Niall -> nnsk1
Lu-Gang, Yu -> Yu, Lu-Gang

In [28]:
df.head()

,last_name,first_name,projects_UG,projects_PG,max_projects,username,email
0,Antonyuk,Svetlana,3.0,1.0,4,antonyuk,NaN
1,Barclay,Jeff,3.0,1.0,4,barclayj,NaN
2,Barsukov,Igor,3.0,1.0,4,igb2,NaN
3,Byrne,Dominic,1.0,1.0,2,rbyrne,NaN
4,Canniffe,Daniel,3.0,1.0,4,canniffe,NaN


In [29]:
df.to_csv("supervisors_3.csv", index=False)